In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\abhis\\OneDrive\\Desktop\\Oil_Retail\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\abhis\\OneDrive\\Desktop\\Oil_Retail'

In [6]:
import pandas as pd 
import json
import numpy as np 
from xgboost import XGBRegressor
import matplotlib.pyplot as plt

In [7]:
df = pd.read_csv(r"C:\Users\abhis\OneDrive\Desktop\Oil_Retail\data\oil_retail_history.csv")
df.head()

,date,price,cost,comp1_price,comp2_price,comp3_price,volume
0,2023-01-01,97.66,86.46,98.39,99.10,97.37,13513
1,2023-01-02,96.46,87.25,95.37,96.25,97.38,13605
2,2023-01-03,94.67,86.40,94.41,94.69,94.50,16540
3,2023-01-04,97.32,87.51,99.12,96.90,97.54,14927
4,2023-01-05,94.08,86.38,93.31,94.61,93.85,13332


In [8]:
df.shape 

(730, 7)

In [9]:
df.dtypes

date            object
price          float64
cost           float64
comp1_price    float64
comp2_price    float64
comp3_price    float64
volume           int64
dtype: object

In [10]:
df.isnull().sum()

date           0
price          0
cost           0
comp1_price    0
comp2_price    0
comp3_price    0
volume         0
dtype: int64

In [11]:
df["date"] = pd.to_datetime(df["date"])

Customers choose where to buy petrol by comparing prices If our companys price is higher than competitors → volume decreases lower than competitors → volume increases same as competitors → normal volume Being more expensive → volume drops Being cheaper → volume increases

So relative price, NOT just the absolute price, affects demand.This is called cross-price elasticity in economics.

In [12]:
df['gap_comp1'] = df['price'] - df['comp1_price']
df['gap_comp2'] = df['price'] - df['comp2_price']
df['gap_comp3'] = df['price'] - df['comp3_price']

In [13]:
df.shape

(730, 10)

In [14]:
df.head()

,date,price,cost,comp1_price,comp2_price,comp3_price,volume,gap_comp1,gap_comp2,gap_comp3
0,2023-01-01,97.66,86.46,98.39,99.10,97.37,13513,-0.73,-1.44,0.29
1,2023-01-02,96.46,87.25,95.37,96.25,97.38,13605,1.09,0.21,-0.92
2,2023-01-03,94.67,86.40,94.41,94.69,94.50,16540,0.26,-0.02,0.17
3,2023-01-04,97.32,87.51,99.12,96.90,97.54,14927,-1.80,0.42,-0.22
4,2023-01-05,94.08,86.38,93.31,94.61,93.85,13332,0.77,-0.53,0.23


In [15]:
df['price_lag_1'] = df['price'].shift(1)
df['volume_lag_1'] = df['volume'].shift(1)

In [16]:
df['volume_ma_7'] = df['volume'].rolling(window=7).mean()
df['volume_ma_30'] = df['volume'].rolling(window=30).mean()

In [17]:
df.isnull().sum()

date             0
price            0
cost             0
comp1_price      0
comp2_price      0
comp3_price      0
volume           0
gap_comp1        0
gap_comp2        0
gap_comp3        0
price_lag_1      1
volume_lag_1     1
volume_ma_7      6
volume_ma_30    29
dtype: int64

In [18]:
df = df.dropna()

In [19]:
from sklearn.model_selection import train_test_split
import joblib

In [23]:
target = "volume"

features = [
    'price', 'cost',
    'comp1_price', 'comp2_price', 'comp3_price',
    'gap_comp1', 'gap_comp2', 'gap_comp3',
    'price_lag_1', 'volume_lag_1',
    'volume_ma_7', 'volume_ma_30'
]

X = df[features]
y = df[target]


In [24]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

X_train.shape, X_test.shape

((560, 12), (141, 12))

In [25]:
model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42
)

model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=300, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

In [26]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from xgboost import XGBRegressor
import joblib

In [27]:
y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = mean_absolute_percentage_error(y_test, y_pred) * 100

rmse, mape


(922.861013818889, 5.2722760467078)

In [28]:
joblib.dump(model, "xgboost_fuel_model.joblib")

['xgboost_fuel_model.joblib']

In [29]:
loaded_model = joblib.load("xgboost_fuel_model.joblib")

In [30]:
import json

def recommend_price(model, today_data, price_range=np.arange(85, 110, 0.1)):
    results = []

    for p in price_range:
        row = today_data.copy()
        row['price'] = p
        
        # price gaps
        row['gap_comp1'] = row['price'] - row['comp1_price']
        row['gap_comp2'] = row['price'] - row['comp2_price']
        row['gap_comp3'] = row['price'] - row['comp3_price']

        # lag + MA
        row['price_lag_1'] = row['price']
        row['volume_lag_1'] = df['volume'].iloc[-1]
        row['volume_ma_7'] = df['volume'].tail(7).mean()
        row['volume_ma_30'] = df['volume'].tail(30).mean()

        row_df = pd.DataFrame([row])

        predicted_volume = model.predict(row_df[features])[0]
        profit = (p - row['cost']) * predicted_volume

        results.append([p, predicted_volume, profit])

    results_df = pd.DataFrame(results, columns=['price', 'pred_volume', 'profit'])
    best_row = results_df.loc[results_df['profit'].idxmax()]
    return best_row, results_df


In [31]:
path = "data/today_example.json"

with open(path, "r") as f:
    today_data = json.load(f)

if isinstance(today_data, dict):
    today_data = today_data
else:
    today_data = today_data[0]

In [32]:
best_price, table = recommend_price(loaded_model, today_data)
best_price

price             109.900000
pred_volume     12168.779297
profit         293632.644434
Name: 249, dtype: float64